## Imports and Functions

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt

import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
    polygons,
)


colors = pc.qualitative.D3
angstrom = "\u212B"
gamma = "\u0393"

In [ ]:
def fit_partial_cone(
    x,
    y,
    data,
    xlim,
    ylim,
    centers=[0.25, 0.4],
    num_peaks=2,
    window=0.02,
    steps=20,
    offset_type="quadratic",
    plot=True,
):
    x_fit, y_fit, d_fit = analysis_functions.limit_dataset(
        x, y, data, xlim=(xlim[0], xlim[1]), ylim=(ylim[0], ylim[1])
    )

    # fit top of cone
    if num_peaks == 1:
        fits = []
        coords = []

        for yval in np.linspace(ylim[0], ylim[1], steps):
            row = analysis_functions.get_averaged_slice(
                analysis_functions.get_horizontal_slice(d_fit, y_fit, yval, window),
                axis="y",
            )

            fit = fitting_functions.fit_lorentzian_data(
                x=x_fit,
                data=row,
                num_peaks=num_peaks,
                amplitudes=0.5,
                centers=[centers[0]],
                sigmas=0.05,
                offset_type=offset_type,
            )
            fits.append(fit)
            coords.extend(
                [(fit.best_values[f"i{i}_center"], yval) for i in range(num_peaks)]
            )
        return coords

    if num_peaks == 2:
        fits1 = []
        coords1 = []

        for yval in np.linspace(ylim[0], ylim[1], steps):
            row = analysis_functions.get_averaged_slice(
                analysis_functions.get_horizontal_slice(d_fit, y_fit, yval, window),
                axis="y",
            )
            fit1 = fitting_functions.fit_lorentzian_data(
                x=x_fit,
                data=row,
                num_peaks=num_peaks,
                amplitudes=0.5,
                centers=[centers[0], centers[1]],
                sigmas=0.05,
                offset_type=offset_type,
            )
            fits1.append(fit1)
            coords1.extend(
                [(fit1.best_values[f"i{i}_center"], yval) for i in range(num_peaks)]
            )
        return coords1

    else:
        return ValueError(f"num_peaks is {num_peaks}; must be 1 or 2")

# Load Data

In [ ]:
# ## 1 ML Film (XUV + Lamp) ##

# ## Convert .ibw to .h5 ##

# ddir = r"E:\atully\k-corrected data\Apr_2021\2D_kcorrected"

# # Convert ibw to hdf5
# # fn = r"OMBE_XUV_2D0003__kw.ibw"
# # fn = r"OMBE_XUV_2D0006__kw.ibw"
# fn = r"OMBE_Lamp_2D0006__kw.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, theta, energy = HDF5_loader.load_hdf5(
#     ddir, r"OMBE_Lamp_2D0006__kw.h5"
# )  # load data from hdf5
# data.shape, theta.shape, energy.shape

In [ ]:
## Load Data ##

ddir = r"E:\atully\k-corrected data\Apr_2021\2D_kcorrected"
# file = r"OMBE_XUV_2D0003__kw.h5"
file = r"OMBE_XUV_2D0006__kw.h5"  # xuv band structure
# file = r"OMBE_Lamp_2D0006__kw.h5"  # lamp band structure

data_1ml, kx_1ml, energy_1ml = loading_functions.load_hdf5(ddir, file)

In [ ]:
xaxis_title = f"k<sub>x</sub> ({angstrom}<sup>-1</sup>)"
yaxis_title = f"E<sub>K</sub> (eV)"
# title = f""

In [ ]:
## Plot Data ##

x, y, data = kx_1ml, energy_1ml, data_1ml

fig = tr_functions.thesis_fig(
    title=f"{file}",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(x=x, y=y, z=analysis_functions.norm_data(data), coloraxis="coloraxis")
)

fig.show()

## Fit for EF

In [ ]:
x, y, data = kx_1ml, energy_1ml, data_1ml

xlim = (-0.5, 0.5)
ylim = (18, 18.8)  # EF -- xuv
ylim = (16.5, 17.5)  # EF -- lamp

new_x, new_y, new_data = analysis_functions.limit_dataset(x, y, data, xlim, ylim)

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    title=f"EDC of E<sub>F</sub>",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    dtick_y=0.2,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=new_x,
    y=new_y,
    data=analysis_functions.norm_data(new_data),
    ylims=None,
    x_range=None,
)

# Plot Data
fig.add_trace(go.Scatter(x=y_1d, y=col, name="data", line=dict(color=colors[0])))

fig.show()

In [ ]:
## Fit FD ##
import lmfit as lm

T = 10  # measurement temp
k_B = 8.617333e-5  # eV/K

x = y_1d
data = col

offset_type = "constant"


## FD
def fermi_dirac(x, center, theta, amp):
    arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
    return -amp / 2 * np.tanh(arg)


## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

full_model = lm.models.Model(fermi_dirac) + offset

params = full_model.make_params()

# params["center"].value = 18.3
params["center"].value = 16.9
params["center"].vary = True
# params["center"].min = 1.85
# params["center"].max = 1.95

params["theta"].value = k_B * T
params["theta"].vary = True


for param in params:
    params[param].value = np.float32(params[param].value)
    params[param].min = np.float32(params[param].min)
    params[param].max = np.float32(params[param].max)
    # print(params[param].value.dtype)


# fit = full_model.fit(data, x=x, params=params, nan_policy="propagate")
fit = full_model.fit(data.astype(np.float32), x=x.astype(np.float32), params=params)

fit.plot()

print(f"EF: {fit.params['center'].value}")

## Fit for HOMO

In [ ]:
EF = 18.27  # xuv
# EF = 16.94  # lamp

In [ ]:
xaxis_title = f"k<sub>x</sub> ({angstrom}<sup>-1</sup>)"
yaxis_title = f"E - E<sub>F</sub> (eV)"
# yaxis_title = f"E<sub>K</sub> (eV)"
title = f"{file}"
title = f"1ML C<sub>60</sub> Band Structure<br>(XUV)"
# title = f"1ML C<sub>60</sub> Band Structure<br>(He Lamp)"

In [ ]:
x, y, data = kx_1ml, energy_1ml, data_1ml

xlim = (-0.5, 0.5)
ylim = None  # for EDC

## For fits
# ylim = (14, 18.5)  # Ek -- xuv
# ylim = (12.67, 16.963)  # Ek -- lamp

new_x, new_y, new_data = analysis_functions.limit_dataset(x, y, data, xlim, ylim)

new_y = new_y - EF

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=550,  # lamp
    width=600,
    dtick_y=2,
)

fig.add_trace(
    go.Heatmap(
        x=new_x,
        y=new_y,
        z=analysis_functions.norm_data(new_data),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(colorscale="Blues", reversescale=False)
# fig.show(renderer="svg")

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    # title=f"EDC of C<sub>60</sub> HOMO and HOMO-1",
    # title=f"1ML C<sub>60</sub> EDC (XUV)",
    title=f"EDC",
    yaxis_title=yaxis_title,
    xaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    # dtick_y=0.2,
    height=550,  # lamp
    width=300,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=new_x,
    y=new_y,
    data=analysis_functions.norm_data(new_data),
    ylims=None,
    x_range=None,
)

# Plot Data
fig.add_trace(go.Scatter(x=col, y=y_1d, name="data", line=dict(color=colors[0])))

fig.update_yaxes(range=(np.min(y_1d), np.max(y_1d)))

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    # title=f"EDC of C<sub>60</sub> HOMO and HOMO-1",
    # title=f"1ML C<sub>60</sub> EDC (XUV)",
    title=f"1ML C<sub>60</sub> EDC (He Lamp)",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    dtick_y=0.2,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=new_x,
    y=new_y,
    data=analysis_functions.norm_data(new_data),
    ylims=None,
    x_range=None,
)

# Plot Data
fig.add_trace(go.Scatter(x=y_1d, y=col, name="data", line=dict(color=colors[0])))

## For After Fit
fig.add_trace(go.Scatter(x=y_1d, y=fit.eval(x=y_1d), name="fit"))

components = fit.eval_components(x=y_1d)
for model_name, model_value in list(components.items())[0:1]:
    fig.add_annotation(
        x=fit.params[f"iA__center"].value,
        y=fit.eval(x=fit.params[f"iA__center"].value) + 0.2,
        # xref="x domain",
        # yref="y domain",
        # x=0.01,
        # y=0.99,
        showarrow=False,
        # text=f'Peak center: {fit.params[f"center"].value:.2f} +/- {fit.params[f"center"].stderr:.4f} mm<br>FWHM: {tr_functions.mm_to_ps(tr_functions.sig_to_fwhm(fit.params["sigma"].value)):.3f} +/- {tr_functions.mm_to_ps(tr_functions.sig_to_fwhm(fit.params[f"sigma"].stderr)):.4f} ps',
        text=f'Peak center:<br>{fit.params[f"iA__center"].value:.2f} +/- {fit.params[f"iA__center"].stderr:.4f} eV<br><br>FWHM:<br>{fit.params[f"iA__fwhm"].value:.4f} +/- {fit.params[f"iA__fwhm"].stderr:.4f} eV',
        font=dict(size=18),
    )

fig.show()

In [ ]:
## Fit Data ##

x = y_1d
data = col

offset_type = "linear"

## Offset
# c = np.mean(data)
c = 0
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

## Gaussian
gauss1 = fitting_functions.make_gaussian(num="A_", amplitude=1, center=-1.69, sigma=0.5)
gauss2 = fitting_functions.make_gaussian(num="B_", amplitude=1, center=-3, sigma=0.5)
gauss_au = fitting_functions.make_gaussian(
    num="C_", amplitude=1, center=-3.7, sigma=0.5
)

# ## Gaussian
# gauss1 = fitting_functions.make_gaussian(num="A_", amplitude=1, center=0, sigma=0.5)
# gauss2 = fitting_functions.make_gaussian(num="B_", amplitude=1, center=-1.3, sigma=0.5)


full_model = gauss1 + gauss2 + gauss_au + offset

fit = full_model.fit(data, x=x)

fit.plot()

print("HOMO")
print(fit.params["iA__center"])
print(fit.params["iA__fwhm"])
print("HOMO-1")
print(fit.params["iB__center"])
print(fit.params["iB__fwhm"])

# fit.params

In [ ]:
fit.params

# HV dependence

In [ ]:
# ## Convert .ibw to .h5 ##

# ddir = r"E:\atully\k-corrected data\Apr_2021\2D_kcorrected"

# # Convert ibw to hdf5
# # fn = r"OMBE_XUV_2D0007__kw.ibw"
# # fn = r"OMBE_XUV_2D0008__kw.ibw"
# # fn = r"OMBE_XUV_2D0009__kw.ibw"
# # fn = r"OMBE_XUV_2D0010__kw.ibw"
# # fn = r"OMBE_XUV_2D0011__kw.ibw"
# # fn = r"OMBE_XUV_2D0012__kw.ibw"
# fn = r"OMBE_XUV_2D0013__kw.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, theta, energy = HDF5_loader.load_hdf5(
#     ddir, r"OMBE_XUV_2D0013__kw.h5"
# )  # load data from hdf5
# data.shape, theta.shape, energy.shape

In [ ]:
# ## Convert .ibw to .h5 ##

# ddir = r"E:\atully\k-corrected data\Jan_2021"

# # Convert ibw to hdf5
# fn = r"OMBE_XUV_2D_2.ibw"

# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, theta, energy = HDF5_loader.load_hdf5(
#     ddir, r"OMBE_XUV_2D_2.h5"
# )  # load data from hdf5
# data.shape, theta.shape, energy.shape

In [ ]:
## Load Data ##

## 1 ML C60/Au(111) hv = 25
ddir = r"E:\atully\k-corrected data\Apr_2021\2D_kcorrected"
file = r"OMBE_XUV_2D0008__kw.h5"  

data_1ml, kx_1ml, energy_1ml = loading_functions.load_hdf5(ddir, file)

In [ ]:
## Load Data ##

## 1 ML C60/Au(111), Au sp band visible (theta motor = -15) hv = 25
ddir = r"E:\atully\k-corrected data\Jan_2021"
file = r"OMBE_XUV_2D_2.h5"

data_au, kx_au, energy_au = loading_functions.load_hdf5(ddir, file)

In [ ]:
xaxis_title = f"k<sub>x</sub> ({angstrom}<sup>-1</sup>)"
yaxis_title = f"E<sub>K</sub> (eV)"

# title = f"hv = 22.7 eV"
title = f"hv = 25 eV"
# title = f"hv = 27.4 eV"
# title = f"hv = 29.8 eV"
# title = f"hv = 32.2 eV"  # scan 11
# title = f"hv = 34.6 eV"  # scan 12
# title = f"hv = 37 eV"  # scan 13

In [ ]:
x, y, data = kx_1ml, energy_1ml, data_1ml
# x, y, data = energy_au, kx_au, data_au.T

xlim = (-0.5, 0.5)
# xlim = None
ylim = None
# ylim = (17.2, 18.5)  # Ek

new_x, new_y, new_data = analysis_functions.limit_dataset(x, y, data, xlim, ylim)

# new_y = new_y - EF

In [ ]:
## Plot Data ##

# x, y, data = kx_1ml, energy_1ml, data_1ml
x, y, data = new_x, new_y, new_data

fig = tr_functions.thesis_fig(
    # title=f"{file}",
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=500,
    width=500,
    # dtick_x=0.2,
)

fig.add_trace(
    go.Heatmap(x=x, y=y, z=analysis_functions.norm_data(data), coloraxis="coloraxis")
)

# x, fit1.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3

## For After Fit
# fig.add_trace(go.Scatter(x=xfit, y=fit_line1.eval(x=xfit), name="fit"))
# fig.add_trace(go.Scatter(x=x, y=fit_line1.eval(x=x), name="fit"))
# fig.add_trace(
#     go.Scatter(
#         x=[c[0] for c in coords_left],
#         y=[c[1] for c in coords_left],
#         name="fit",
#         mode="markers",
#         text=list(range(len(coords_left))),
#     )
# )
##

# fig.update_coloraxes(cmin=0, cmax=0.13)
fig.update_coloraxes(colorscale="Blues", reversescale=False)
fig.update_layout(yaxis_range=(min(y), max(y)))

fig.show()

In [ ]:
xfit, yfit, datafit = new_x, new_y, new_data

coords_left = fit_partial_cone(
    xfit,
    yfit,
    datafit,
    xlim=(-0.35, -0.0),
    ylim=(19.2, 19.7),
    centers=[-0.22],
    num_peaks=1,
    window=0.1,
)  # left line
# coords_bottom = fit_partial_cone(xfit, yfit, datafit, xlim=(-0.38, 0.0), ylim=(17.6, 17.7), centers=[-0.25, -0.1], num_peaks=2, window=0.02)  # bottom of left  cone
# coords_middle = fit_partial_cone(xfit, yfit, datafit, xlim=(-0.3, -0.07), ylim=(17.7, 17.8), centers=[-0.25, -0.1], num_peaks=2, window=0.02)  # middle of left  cone

# x_line, y_line = compile_peaks(coords_top, coords_middle, coords_bottom)  # compile coords of all peaks
x_line, y_line = np.array(
    [c[0] for i, c in enumerate(coords_left) if i not in [0, 3, 4, 13]]
), np.array([c[1] for i, c in enumerate(coords_left) if i not in [0, 3, 4, 13]])
# fit1, fit2 = fit_cone_lines(x_line, y_line)  # linear fits

In [ ]:
from lmfit.models import LinearModel

model = LinearModel()
fit = model.fit(y_line, x=x_line)

In [ ]:
fit.params["slope"].value, fit.params["slope"].stderr

In [ ]:
## Plot Data ##

# x, y, data = kx_1ml, energy_1ml, data_1ml
x, y, data = new_x, new_y, new_data

fig = tr_functions.thesis_fig(
    # title=f"{file}",
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=500,
    width=500,
    dtick_x=0.2,
)

fig.add_trace(
    go.Heatmap(x=x, y=y, z=analysis_functions.norm_data(data), coloraxis="coloraxis")
)

# x, fit1.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3

## For After Fit
# fig.add_trace(go.Scatter(x=xfit, y=fit_line1.eval(x=xfit), name="fit"))
fig.add_trace(
    go.Scatter(
        x=x,
        y=fit.eval(x=x),
        name="fit",
        line=dict(color="purple", dash="dash"),
    )
)
fig.add_trace(
    go.Scatter(
        x=[c[0] for i, c in enumerate(coords_left) if i not in [0, 3, 4, 13]],
        y=[c[1] for i, c in enumerate(coords_left) if i not in [0, 3, 4, 13]],
        name="fit",
        mode="markers",
        text=list(range(len(coords_left))),
        line=dict(color="green"),
    )
)

fig.add_vline(x=-0.35, line=dict(color="grey", dash="dot"))
##

# fig.update_coloraxes(cmin=0, cmax=0.13)
fig.update_coloraxes(colorscale="Blues", reversescale=False)
fig.update_layout(yaxis_range=(min(y), max(y)))

fig.show()

# FS Scans -- different slices

In [ ]:
fp = r"E:\atully\arpes_data\2022_October\k_corrected"
fn = r"FS4_avg4_gkw_filteredFFT_0.00int.h5"  # pristine Au
data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

In [ ]:
ddir = r"E:\atully\arpes_data\2022_October\k_corrected"
files = []
files = [f"FS4_avg4_gkw_filteredFFT_0.00int.h5"]  # pristine Au

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad = ARPES_DATA[files[0]]

In [ ]:
ddir = r"E:\atully\arpes_data\2022_October\k_corrected"
files = []
files = [f"FS1_avg_gkw_filteredFFT_0.00int.h5"]  # pristine Au, bottom of sp band

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad = ARPES_DATA[files[0]]

In [ ]:
ddir = r"E:\atully\k-corrected data\Apr_2021\XUV_FS_gamma0"
files = []
files = [f"XUV_FS_gamma0_gkw11_filteredFFT_0.00int.h5"]  # xuv diffraction FS

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad = ARPES_DATA[files[0]]

In [ ]:
## Integrate over desired angular range ##

slice_dim = "z"
slice_val = -0.36  # C60
# slice_val = -0.72  # Au
slice_val = -1.08  # Au bottom (comp for left side of cone?)
# slice_val = -0.25  # Au for hv=25 comp
int_range = 0.1  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

# Au
xlim = (-1.2, -0.25)
ylim = (15.95, 16.8)

xlim = (-0.6, 0.6)
ylim = (15.95, 16.8)

# XUV 1 ML
# xlim = (-0.6, 0.5)
# ylim = (17.55, 18.3)

# xlim = None
# ylim = None
x_bin = 1
y_bin = 1

In [ ]:
x, y, d = tr_functions.slice_datacube(
    ad_dataclass=ad,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    # ylim=(
    #     ad.energy[57],
    #     ad.energy[1007],
    # ),  # get rid of zero padding on datasets
    ylim=ylim,
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=False,
    plot_data=False,
)

# x_au, y_au, d_au = x, y, d
x_au_bottom, y_au_bottom, d_au_bottom = x, y, d
# x_xuv, y_xuv, d_xuv = x, y, d

In [ ]:
title = f"k<sub>y</sub> = {slice_val} {angstrom}<sup>-1</sup>"

In [ ]:
# x_plot, y_plot, d_plot = x_au, y_au - 16.8, d_au
x_plot, y_plot, d_plot = x_au_bottom, y_au_bottom - 16.8, d_au_bottom
# x_plot, y_plot, d_plot = x_xuv, y_xuv - 18.3, d_xuv

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    # yaxis_title=yaxis_title,
    yaxis_title="E - E<sub>F</sub> (eV)",
    equiv_axes=False,
    height=500,
    width=500,
    dtick_y=0.2,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=analysis_functions.norm_data(d_plot),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0.0, cmax=0.3)

fig.update_coloraxes(colorscale="Blues", reversescale=False)
fig.update_layout(
    yaxis_range=(min(y_plot), max(y_plot)), xaxis_range=(min(x_plot), max(x_plot))
)
# fig.update_layout(
#     yaxis_range=(min(y_plot), max(y_plot)), xaxis_range=(min(x_plot), -0.2)
# )

fig.show()

In [ ]:
slice_dim = "y"
# v = 18.3 - 0.1
v = 18.38 - 0.5  # 0.1, 0.3, 0.5
int_range = 0.2
xlim = (-0.73, 0.52)
ylim = (-1.4, 0.1)
# ylim = None
x_bin = 1
y_bin = 1


# Get Slice

x, y, d = tr_functions.slice_datacube(
    ad_dataclass=ad,
    slice_dim=slice_dim,
    slice_val=v,
    int_range=int_range,
    xlim=xlim,
    ylim=ylim,
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=False,
    plot_data=False,
)

## Plot Data Plotly
fig = tr_functions.thesis_fig(
    title=f"E<sub>F</sub>",
    xaxis_title=xaxis_title,
    yaxis_title=f"k<sub>y</sub> ({angstrom}<sup>-1</sup>)",
    equiv_axes=True,
    height=500,
    width=500,
    dtick_y=0.4,
    dtick_x=0.4,
)

fig.add_trace(go.Heatmap(x=x, y=y, z=d, coloraxis="coloraxis"))

# yval = -0.36
# fig.add_shape(
#     go.layout.Shape(
#         type="line",
#         x0=-0.5,
#         x1=0.5,
#         y0=yval,
#         y1=yval,
#         line=dict(color="grey", dash="dash", width=2),
#     )
# )

# fig.update_yaxes(scaleanchor="x", scaleratio=1)

if xlim is not None:
    fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")


# fig.update_coloraxes(cmin=0, cmax=1)
fig.update_coloraxes(colorscale="Blues", reversescale=False)
fig.show()

# Fit Denoised Data

In [ ]:
def load_denoised_data(file_path: str, filename: str):
    data_path = os.path.join(file_path, filename)
    df = pd.read_csv(data_path, skiprows=3, header=None, sep="\s+", skipfooter=2)
    data = np.array(df).T
    with open(data_path, "r") as f:
        lines = f.readlines()
    last = lines[-1]

    x_start, x_step = [
        float(v) for v in re.search("x\s*(-?\d*.\d+),\s*(-?\d+.\d+)", last).groups()
    ]
    y_start, y_step = [
        float(v) for v in re.search("y\s*(-?\d*.\d+),\s*(-?\d+.\d+)", last).groups()
    ]

    x = np.linspace(x_start, x_start + data.shape[1] * x_step, data.shape[1])
    y = np.linspace(y_start, y_start + data.shape[0] * y_step, data.shape[0])
    return x, y, data

In [ ]:
def fit_partial_cone(
    x,
    y,
    data,
    xlim,
    ylim,
    centers=[0.25, 0.4],
    num_peaks=2,
    window=0.02,
    steps=20,
    offset_type="quadratic",
    plot=True,
):
    x_fit, y_fit, d_fit = analysis_functions.limit_dataset(
        x, y, data, xlim=(xlim[0], xlim[1]), ylim=(ylim[0], ylim[1])
    )

    # fit top of cone
    if num_peaks == 1:
        fits = []
        coords = []

        for yval in np.linspace(ylim[0], ylim[1], steps):
            row = analysis_functions.get_averaged_slice(
                analysis_functions.get_horizontal_slice(d_fit, y_fit, yval, window),
                axis="y",
            )

            fit = fitting_functions.fit_lorentzian_data(
                x=x_fit,
                data=row,
                num_peaks=num_peaks,
                amplitudes=0.5,
                centers=[centers[0]],
                sigmas=0.05,
                offset_type=offset_type,
            )
            fits.append(fit)
            coords.extend(
                [(fit.best_values[f"i{i}_center"], yval) for i in range(num_peaks)]
            )
        return coords

    if num_peaks == 2:
        fits1 = []
        coords1 = []

        for yval in np.linspace(ylim[0], ylim[1], steps):
            row = analysis_functions.get_averaged_slice(
                analysis_functions.get_horizontal_slice(d_fit, y_fit, yval, window),
                axis="y",
            )
            fit1 = fitting_functions.fit_lorentzian_data(
                x=x_fit,
                data=row,
                num_peaks=num_peaks,
                amplitudes=0.5,
                centers=[centers[0], centers[1]],
                sigmas=0.05,
                offset_type=offset_type,
            )
            fits1.append(fit1)
            coords1.extend(
                [(fit1.best_values[f"i{i}_center"], yval) for i in range(num_peaks)]
            )
        return coords1

    else:
        return ValueError(f"num_peaks is {num_peaks}; must be 1 or 2")

In [ ]:
def compile_peaks(coords_top, coords_middle, coords_bottom):
    x_line1, y_line1 = np.array([c[0] for c in coords_top]), np.array(
        [c[1] for c in coords_top]
    )

    x_line2 = np.append(x_line1, np.array([c[0] for c in coords_middle]), axis=0)
    x_line_all = np.append(x_line2, np.array([c[0] for c in coords_bottom]), axis=0)

    y_line2 = np.append(y_line1, np.array([c[1] for c in coords_middle]), axis=0)
    y_line_all = np.append(y_line2, np.array([c[1] for c in coords_bottom]), axis=0)

    return x_line_all, y_line_all

In [ ]:
def fit_cone_lines(x_line, y_line, cone_center=None):
    if cone_center is None:
        cone_center = np.mean(x_line)
    # Fits all data points (RIGHT SIDE OF CONE)
    fit_line1 = fitting_functions.fit_linear_data(
        x=x_line[np.where(x_line > cone_center)],
        data=y_line[np.where(x_line > cone_center)],
        num=1,
        aes=1,
        bes=1,
        offset_type=None,
    )

    # Fits all data points (LEFT SIDE OF CONE)
    fit_line2 = fitting_functions.fit_linear_data(
        x=x_line[np.where(x_line < cone_center)],
        data=y_line[np.where((x_line < cone_center))],
        num=1,
        aes=1,
        bes=1,
        offset_type=None,
    )

    return fit_line1, fit_line2

In [ ]:
import pandas as pd
import re

# Denoised Data
fp_dn = r"C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\xuv_ky=-0.36"
fn_dn = "xuv_ky=-0.36_dn.itx"

x_dn, y_dn, data_dn = load_denoised_data(fp_dn, fn_dn)

EF = 18.3
y_dn = y_dn - EF

d_dn = analysis_functions.norm_data(data_dn)

In [ ]:
fig = tr_functions.thesis_fig(
    title=f"{v} eV",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=500,
    width=500,
    dtick_y=0.4,
)

fig.add_trace(go.Heatmap(x=x_dn, y=y_dn, z=d_dn, coloraxis="coloraxis"))

fig.update_coloraxes(colorscale="Blues", reversescale=False)
fig.show()

In [ ]:
xfit, yfit, datafit = x_dn, y_dn, data_dn

coords_top = fit_partial_cone(
    xfit,
    yfit,
    datafit,
    xlim=(-0.4, 0.0),
    ylim=(18.1 - EF, 18.2 - EF),
    centers=[-0.2],
    num_peaks=1,
    window=0.05,
)  # top of left cone
coords_bottom = fit_partial_cone(
    xfit,
    yfit,
    datafit,
    xlim=(-0.38, 0.0),
    ylim=(17.6 - EF, 17.7 - EF),
    centers=[-0.25, -0.1],
    num_peaks=2,
    window=0.02,
)  # bottom of left  cone
coords_middle = fit_partial_cone(
    xfit,
    yfit,
    datafit,
    xlim=(-0.3, -0.07),
    ylim=(17.7 - EF, 17.8 - EF),
    centers=[-0.25, -0.1],
    num_peaks=2,
    window=0.02,
)  # middle of left  cone

x_line, y_line = compile_peaks(
    coords_top, coords_middle, coords_bottom
)  # compile coords of all peaks

In [ ]:
fit1, fit2 = fit_cone_lines(x_line, y_line)  # linear fits

In [ ]:
def reflect_line(y, m, b, x_reflection):
    # Create an array of x values
    x = np.linspace(min(y), max(y), num=len(y))

    # Calculate the y values for the original line
    original_line = m * x + b

    # Calculate the x values for the reflected line
    reflected_x = 2 * x_reflection - x

    # Combine the reflected x values with the original y values
    reflected_line = m * reflected_x + b

    return reflected_x, reflected_line

In [ ]:
x_reflection = -0.21  # Replace with your desired reflection point
m = fit1.params["i0_slope"].value
b = fit1.params["i0_intercept"].value
y = x_dn  # Example y values

reflected_x, reflected_line = reflect_line(y, m, b, x_reflection)

In [ ]:
cone_center = np.mean(x_line)
# Gets all data points (RIGHT SIDE OF CONE)
coords_right_x = x_line[np.where(x_line > cone_center)]
coords_right_y = y_line[np.where(x_line > cone_center)]

In [ ]:
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=500,
    width=500,
    dtick_y=0.4,
    showscale=False,
)

fig.add_trace(go.Heatmap(x=x_dn, y=y_dn, z=d_dn, coloraxis="coloraxis"))

fig.add_trace(
    go.Scatter(
        x=coords_right_x,
        y=coords_right_y,
        mode="markers",
        marker=dict(size=3, color="turquoise")
        # line=dict(color="hotpink"),
    )
)

fig.add_trace(
    go.Scatter(x=x_dn, y=fit1.eval(x=x_dn), line=dict(color="purple", dash="dash"))
)

# fig.add_vline(x=-0.21, line=dict(color="grey", dash="dot"))

fig.update_coloraxes(colorscale="Blues", reversescale=False)
fig.update_layout(yaxis_range=(min(y_dn), 0), xaxis_range=(min(x_dn), max(x_dn)))
fig.show()

In [ ]:
x_plot, y_plot, d_plot = x_xuv, y_xuv - EF, d_xuv

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    # yaxis_title=yaxis_title,
    yaxis_title="E - E<sub>F</sub> (eV)",
    equiv_axes=False,
    height=500,
    width=500,
    dtick_y=0.4,
    dtick_x=0.5,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=analysis_functions.norm_data(d_plot),
        coloraxis="coloraxis",
    )
)

## For After Fit

fig.add_trace(
    go.Scatter(x=x_plot, y=fit1.eval(x=x_plot), line=dict(color="purple", dash="dash"))
)

fig.update_coloraxes(cmin=0.0, cmax=0.5)

fig.update_coloraxes(colorscale="Blues", reversescale=False)
fig.update_layout(
    yaxis_range=(min(y_plot), max(y_plot)), xaxis_range=(-0.5, max(x_plot))
)
# fig.update_layout(
#     yaxis_range=(min(y_plot), max(y_plot)), xaxis_range=(min(x_plot), -0.2)
# )

fig.show()

In [ ]:
x_plot, y_plot, d_plot = x_au, y_au - 16.8, d_au

fig = tr_functions.thesis_fig(
    title=f"k<sub>y</sub> = -0.72 {angstrom}<sup>-1</sup>",
    xaxis_title=xaxis_title,
    # yaxis_title=yaxis_title,
    yaxis_title="E - E<sub>F</sub> (eV)",
    equiv_axes=False,
    height=500,
    width=500,
    dtick_y=0.4,
    dtick_x=0.5,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=analysis_functions.norm_data(d_plot),
        coloraxis="coloraxis",
    )
)
## For After Fit

offset = -0.72 * 6.87

fig.add_trace(
    go.Scatter(
        x=x_plot, y=fit1.eval(x=x_plot) + offset, line=dict(color="purple", dash="dash")
    )
)

## expected EF value
fig.add_trace(
    go.Scatter(
        x=[-0.83],
        y=[0],  # 16.772 - 16.8
        # name="fit",
        mode="markers",
        marker=dict(size=[20], color="grey"),
    )
)


fig.update_coloraxes(cmin=0.0, cmax=0.5)

fig.update_coloraxes(colorscale="Blues", reversescale=False)
fig.update_layout(
    yaxis_range=(-0.75, max(y_plot)), xaxis_range=(min(x_plot), max(x_plot))
)
# fig.update_layout(
#     yaxis_range=(min(y_plot), max(y_plot)), xaxis_range=(min(x_plot), -0.2)
# )

fig.show()

In [ ]:
fit1

In [ ]:
from lmfit.models import LinearModel

model = LinearModel()
fit1_werr = model.fit(coords_right_y, x=coords_right_x)

fit1_werr.params["slope"].value, fit.params["slope"].stderr

In [ ]:
fit1_werr